In [ ]:
LR = 0.001
EPOCHS = 25
NUM_CLASS = 10
BATCH_SIZE = 16
device = "cuda"
PATH = '/content/drive/MyDrive/APA/Assigment1/Parte2'
img_size = (28, 28)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir data
!mkdir PATH

## Importação do DataSet

In [ ]:
import torch
from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))
     ])

# transform = tt.Compose([
#     tt.RandomResizedCrop(224),
#     tt.RandomHorizontalFlip(),
#     tt.RandomRotation(30),
#     tt.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
#     tt.ToTensor(),
#     tt.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])

batch_size = 64
# Download and load the training data
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

# Download and load the test data
testset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)




## Criação do Modelo

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Autoencoder(nn.Module):
    def __init__(self,img_size):
        super(Autoencoder, self).__init__()
        # Encoder
        self.encoder_conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding=1)
        self.encoder_conv2 = nn.Conv2d(in_channels=8, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.encoder_fc1 = nn.Linear(in_features=32 * img_size[0]*img_size[1], out_features=128)  # Assuming input size is 28x28

        # Decoder
        self.decoder_fc1 = nn.Linear(in_features=128, out_features=32 * img_size[0]*img_size[1])
        self.decoder_conv1 = nn.Conv2d(in_channels=32, out_channels=8, kernel_size=3, stride=1, padding=1)
        self.decoder_conv2 = nn.Conv2d(in_channels=8, out_channels=1, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        # Encoder
        x = F.elu(self.encoder_conv1(x))
        x = F.elu(self.encoder_conv2(x))
        x = x.view(x.size(0), -1)  # Flatten before fully connected layer
        x = F.elu(self.encoder_fc1(x))

        # Decoder
        x = F.elu(self.decoder_fc1(x))
        x = x.view(x.size(0), 32, 28, 28)  # Reshape before transposed convolutions
        x = F.elu(self.decoder_conv1(x))
        x = torch.sigmoid(self.decoder_conv2(x))  # Sigmoid activation for the last layer

        return x


# Training

In [ ]:
import os
import torch.optim as optim

def train(model, loss_fn, optimizer,scheduler,training_loader, num_epoch,path,modelName,device="cuda"):

  os.makedirs(f"{PATH}/{path}", exist_ok=True)

  for epoch in range(num_epoch):

    training_loss = 0.0
    correct = 0
    num_examples = 0
    num_correct = 0
    model.train()

    for batch_id, data in enumerate(training_loader, 0):
      inputs, labels = data
      inputs = inputs.to(device)
      labels = labels.to(device)
      output = model(inputs)

      loss = loss_fn(output, labels)

      optimizer.zero_grad()

      loss.backward()

      optimizer.step()

      training_loss += loss.item()

      preds = torch.argmax(output, dim=1)
      correct = preds.eq(labels)
      num_correct += correct.sum().item()
      num_examples += correct.shape[0]

    accuracy = num_correct / num_examples
    training_loss /= len(train_loader)
    scheduler.step()

    print('Epoch: {}, Training Loss: {:.4f}, Acc: {:.3f}'.format(epoch, training_loss,accuracy*100))

    torch.save(model, f"{PATH}/{path}/{modelName}_epoch={epoch}") #Gravar o modelo no Disco

In [ ]:
model = Autoencoder([28,28])
# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
autoEncoder = model.to(device)